In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q accelerate>=0.16.0 torchvision datasets diffusers tensorboard bitsandbytes

In [ ]:
!accelerate config default

In [ ]:
!git clone https://github.com/docty/diffuser-training.git

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

In [ ]:
import shutil
shutil.rmtree('/content/dog/.cache')

In [ ]:
!accelerate launch /content/diffuser-training/train_dreambooth.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4"  \
  --instance_data_dir="dog" \
  --class_data_dir="path-to-class-images" \
  --output_dir="dreambooth-model" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of sks dog" \
  --class_prompt="a photo of dog" \
  --resolution=128 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=2 --gradient_checkpointing \
  --use_8bit_adam \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=800 \
  --push_to_hub

# Inference

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
model_id = "Docty/dreambooth-model"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")


In [ ]:
prompt = "A photo of   pipe sitting under the sea"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

#image.save("dog-bucket.png")
image